# HM3D Waypoint Visualization for Objectnav

### Purpose
An example of how training data for trajectory generation by a VLM is created using the [Habitat-Matterport 3D dataset](https://matterport.com/partners/meta)

## Set up simulation
- Load a scene from hm3d_minival_v0.2 (includes semantic annotations)
- Set up agent sensors

In [1]:
import habitat_sim
import magnum as mn
import numpy as np

In [2]:
scene_graph = habitat_sim.SceneGraph()

backend_cfg = habitat_sim.SimulatorConfiguration()
backend_cfg.scene_id = "data/ovon/versioned_data/hm3d-0.2/hm3d/train/00250-U3oQjwTuMX8/U3oQjwTuMX8.basis.glb"
backend_cfg.scene_dataset_config_file = "data/ovon/versioned_data/hm3d-0.2/hm3d/hm3d_annotated_basis.scene_dataset_config.json"
#backend_cfg.load_semantic_mesh = True
sensor_height = 0.5

rgb_cfg = habitat_sim.CameraSensorSpec()
rgb_cfg.uuid = "color"
rgb_cfg.sensor_type = habitat_sim.SensorType.COLOR
#rgb_cfg.hfov = mn.Deg(120)
rgb_cfg.resolution = [1024, 1024]
rgb_cfg.position = [0.0, sensor_height, 0.0]  # [x, y, z] relative to agent

final_cfg = habitat_sim.CameraSensorSpec()
final_cfg.uuid = "final_color"
final_cfg.sensor_type = habitat_sim.SensorType.COLOR
final_cfg.hfov = mn.Deg(120)
final_cfg.resolution = [1024, 1024]
final_cfg.position = [0.0, sensor_height, 0.0]  # [x, y, z] relative to agent

# Semantic sensor
sem_cfg = habitat_sim.CameraSensorSpec()
sem_cfg.uuid = "semantic"
sem_cfg.sensor_type = habitat_sim.SensorType.SEMANTIC
#sem_cfg.hfov = mn.Deg(120)
sem_cfg.resolution = [1024, 1024]
sem_cfg.position = [0.0, sensor_height, 0.0]
#

# Depth sensor
depth_cfg = habitat_sim.CameraSensorSpec()
depth_cfg.uuid = "depth"
depth_cfg.sensor_type = habitat_sim.SensorType.DEPTH
#depth_cfg.hfov = mn.Deg(120)
depth_cfg.resolution = [1024, 1024]
depth_cfg.position = [0.0, sensor_height, 0.0]
#

agent_cfg = habitat_sim.agent.AgentConfiguration()
agent_cfg.sensor_specifications = [rgb_cfg, sem_cfg, depth_cfg, final_cfg]

sim_cfg = habitat_sim.Configuration(backend_cfg, [agent_cfg])

In [4]:
sim = habitat_sim.Simulator(sim_cfg)

### Load navmesh

In [12]:
import numpy as np

In [5]:
import numpy as np

navmesh_file = "data/ovon/versioned_data/hm3d-0.2/hm3d/train/00250-U3oQjwTuMX8/U3oQjwTuMX8.basis.navmesh"

sim.pathfinder.load_nav_mesh(navmesh_file)
print("Navigation mesh loaded successfully!")

# Get a random navigable point to start
start_point = sim.pathfinder.get_random_navigable_point()
agent_state = habitat_sim.AgentState()
agent_state.position = start_point
agent_state.rotation = np.quaternion(1, 0, 0, 0)  # Identity quaternion
sim.get_agent(0).set_state(agent_state)

Navigation mesh loaded successfully!


In [6]:
import numpy as np
import habitat_sim
import matplotlib.pyplot as plt
import cv2
import magnum as mn
import traceback

def get_2d_point(sim, render_camera, point_3d):
    # use the camera and projection matrices to transform the point onto the near plane
    projected_point_3d = render_camera.projection_matrix.transform_point(
        render_camera.camera_matrix.transform_point(point_3d)
    )
    # convert the 3D near plane point to integer pixel space
    point_2d = mn.Vector2(projected_point_3d[0], -projected_point_3d[1])
    point_2d = point_2d / render_camera.projection_size()[0]
    point_2d += mn.Vector2(0.5)
    point_2d *= render_camera.viewport
    return mn.Vector2i(point_2d)

## Function to check if a 3D point is visible from the current camera view
To do this:  
    - Finds 2D projection of 3D point onto depth camera view  
    - Checks depth at those coordinates  
    - If depth is >= straight line distance to point, it's visible

I don't know if this is actually the best way to do this -- I tried using habitat_sim.geo.Ray and checking for collisions but that didn't work well. The depth camera approach works at least.

In [7]:
import numpy as np
import habitat_sim
from habitat_sim.utils.common import quat_from_angle_axis

def check_point_visibility_depth(sim, observer_pos, target_pos, observer_rotation=None):
    """
    Checks for point visibility using the depth buffer for accurate occlusion.

    Args:
        sim: The Habitat simulator instance.
        observer_pos (np.ndarray): Position of the observer (camera).
        target_pos (np.ndarray): Position of the target point to check.
        observer_rotation (quaternion): Rotation quaternion of the observer.

    Returns:
        bool: True if the target is visible, False otherwise.
    """
    agent = sim.get_agent(0)
    
    agent_state = agent.get_state()
    agent_state.position = observer_pos
    if observer_rotation is not None:
        agent_state.rotation = observer_rotation
    agent.set_state(agent_state)
    
    observations = sim.get_sensor_observations()
    
    depth_sensor = agent._sensors.get("depth")
        
    render_camera = depth_sensor.render_camera
    
    target_vec3 = mn.Vector3(target_pos[0], target_pos[1], target_pos[2])
    
    camera_space_point = render_camera.camera_matrix.transform_point(target_vec3)
    
    if camera_space_point.z > 0:
        return False

    point_2d = get_2d_point(sim, render_camera, target_vec3)
    
    depth_map = observations["depth"]
    height, width = depth_map.shape[:2]
    
    if not (0 <= point_2d.x < width and 0 <= point_2d.y < height):
        return False
    
    depth_at_pixel = depth_map[point_2d.y, point_2d.x]
    
    # Calculate the straight-line distance from camera to target
    # This is the actual 3D Euclidean distance
    camera_to_target_distance = np.linalg.norm(target_pos - observer_pos)
    
    tolerance = 0.01  #1cm
    
    is_visible = depth_at_pixel >= (camera_to_target_distance - tolerance)
    
    return is_visible

## Function to upsample waypoints along a shortest path
Waypoints along the shortest path to a target position are often sparse. This presents some issues:  
- Sometimes the only visible point in a trajectory is far away, which (imo) decreases how valuable that sample is for model training by decreasing the amount of points the model will use as context to improve understanding of proper trajectory generation  
    
- Often, when we go around a corner or down stairs, we end up with a path between two points with *no visible intermediate points*, which wrecks the continuity of a multi-turn trajectory generation problem for the model  
  
Take the following as an example:  
<img src = "traj-vis-prob.png">

We came from the path marked by the dotted yellow line, and are going to the red waypoint on the staircase. However, this waypoint is obstructed, which means we have no trajectory to generate.  

To solve this, the `upsample_trajectory_waypoints` function is used to generate the blue points.

In [8]:
import math

def upsample_trajectory_waypoints(shortest_path, max_dist = 1):
    """
    Function for upsampling the number of waypoints along a path.
    Parameters:
        -shortest_path: List of Magnum Vector3 (habitat_sim.nav.ShortestPath.points)
        -max_dist (optional): Maximum distance between waypoints, default 1 meter
    """
    sp = shortest_path.points
    pointList = []
    
    for v1, v2 in zip(sp, sp[1:]):
        d = (v1 - v2).length()
        
        pointList.append(v1)
        
        if d > max_dist:
            segments = math.ceil(d / max_dist)
            
            #Parametric form of the line between v1 and v2
            #L(t) = v1 + t(v2 - v1)
            
            for i in range(1, segments):
                t = i / segments
                p = v1 + (t * (v2 - v1))
                pointList.append(p)
    
    pointList.append(sp[-1])
    
    return pointList

## Function to get visible trajectory at a waypoint as 2D coordinates in camera space

In [9]:
def calculate_rotation_to_face_direction(direction_vector):
    """Calculate quaternion rotation to face a given direction"""
    direction_2d = np.array([direction_vector[0], direction_vector[2]])
    
    if np.linalg.norm(direction_2d) < 1e-6:
        return np.quaternion(1, 0, 0, 0)
    
    direction_2d = direction_2d / np.linalg.norm(direction_2d)
    angle = np.arctan2(-direction_2d[0], -direction_2d[1])
    
    cos_half_angle = np.cos(angle / 2)
    sin_half_angle = np.sin(angle / 2)
    
    rotation = np.quaternion(cos_half_angle, 0, sin_half_angle, 0)
    return rotation

In [10]:
def get_visible_trajectory(sim, shortest_path, current_waypoint_index, return_plain_view=True, return_depth_map=False):
    """
    Get camera space coordinates of all visible future trajectory waypoints
    
    Parameters:
        - sim: habitat sim instance
        - shortest_path: habitat_sim.nav.ShortestPath
        - current_waypoint_index: int
    
    Returns:
        - visible_waypoints_info: List of tuples (waypoint_index, (x, y)) for visible waypoints
        - rgb_img: RGB image (if return_plain_view is True)
    """
    
    if current_waypoint_index >= len(shortest_path.points):
        return [], None if return_plain_view else []
    
    current_pos = shortest_path.points[current_waypoint_index]
    agent = sim.get_agent(0)
    agent_state = agent.get_state()
    agent_state.position = current_pos
    
    # Calculate rotation to face next waypoint if available
    if current_waypoint_index < len(shortest_path.points) - 1:
        next_waypoint = shortest_path.points[current_waypoint_index + 1]
        direction_vector = next_waypoint - current_pos
        if np.linalg.norm(direction_vector) > 0.01:
            agent_state.rotation = calculate_rotation_to_face_direction(direction_vector)
    
    agent.set_state(agent_state)
    
    render_camera = agent._sensors.get("color").render_camera
    
    observations = sim.get_sensor_observations()
    rgb_img = observations["color"].copy()
    if return_depth_map:
        depth_img = observations["depth"].copy()
    
    visible_waypoints_info = []
    
    for i in range(current_waypoint_index + 1, len(shortest_path.points)):
        if i == current_waypoint_index:
            continue
            
        waypoint = shortest_path.points[i]
        
        observer_rotation = agent_state.rotation
        
        is_visible = check_point_visibility_depth(sim, current_pos, waypoint, observer_rotation)
        
        if is_visible:
            try:
                point_2d = get_2d_point(sim, render_camera, mn.Vector3(waypoint))
                
                if 0 <= point_2d.x < rgb_cfg.resolution[0] and 0 <= point_2d.y < rgb_cfg.resolution[1]:
                    visible_waypoints_info.append((i, (point_2d.x, point_2d.y)))
            except Exception as e:
                print(f"Projection failed for waypoint {i}: {e}")
    
    #print(visible_waypoints_info)
    
    if len(visible_waypoints_info) == 0 and current_waypoint_index < len(shortest_path.points) - 1:
        next_waypoint = shortest_path.points[current_waypoint_index + 1]
        generated_waypoint = generate_visible_waypoint_on_line(
            sim, current_pos, next_waypoint, agent_state.rotation, render_camera
        )
        if generated_waypoint:
            #print("~~~~~GENERATED WAYPOINT~~~~~~~~")
            visible_waypoints_info.append((current_waypoint_index + 1, generated_waypoint))
    
    if return_plain_view:
        if return_depth_map:
            return [visible_waypoints_info[-1]], rgb_img, depth_img
        return [visible_waypoints_info[-1]], rgb_img
    else:
        return [visible_waypoints_info[-1]]

## Function to generate a visible waypoint when we can't see one
Sometimes, as a product of camera height and FOV, we can't see a point if it's right below us. This function will generate a corresponding point at the bottom edge of the camera for these cases.

In [11]:
def generate_visible_waypoint_on_line(sim, start_pos, end_pos, observer_rotation, render_camera):
    """
    Generate a waypoint at the edge of the camera view in the direction of the target waypoint.
    If the target is off-screen, project a point at the camera boundary in that direction.
    
    Parameters:
        - sim: habitat sim instance
        - start_pos: starting position (current waypoint)
        - end_pos: ending position (target waypoint)
        - observer_rotation: rotation of the observer
        - render_camera: camera for 2D projection
    
    Returns:
        - (x, y): 2D coordinates of the generated visible waypoint at camera edge, or None if failed
    """
    try:
        # First, try to project the target waypoint to see where it would be
        target_2d = get_2d_point(sim, render_camera, mn.Vector3(end_pos))
        
        # Camera bounds
        width, height = rgb_cfg.resolution[0], rgb_cfg.resolution[1]
        
        # If the point is already in bounds, something went wrong (shouldn't call this function)
        if 0 <= target_2d.x < width and 0 <= target_2d.y < height:
            return (target_2d.x, target_2d.y)
        
        center_x, center_y = width / 2, height / 2
        
        dir_x = target_2d.x - center_x
        dir_y = target_2d.y - center_y
        
        # Normalize the direction
        dir_length = np.sqrt(dir_x**2 + dir_y**2)
        if dir_length < 1e-6:  # Avoid division by zero
            return None
            
        dir_x /= dir_length
        dir_y /= dir_length
        
        # Find intersection with camera boundaries
        # Check intersection with each edge and pick the closest one
        
        # Right edge (x = width - 1)
        if dir_x > 0:
            t_right = (width - 1 - center_x) / dir_x
            edge_y = center_y + t_right * dir_y
            if 0 <= edge_y < height:
                return (width - 1, int(edge_y))
        
        # Left edge (x = 0)
        if dir_x < 0:
            t_left = -center_x / dir_x
            edge_y = center_y + t_left * dir_y
            if 0 <= edge_y < height:
                return (0, int(edge_y))
        
        # Bottom edge (y = height - 1)
        if dir_y > 0:
            t_bottom = (height - 1 - center_y) / dir_y
            edge_x = center_x + t_bottom * dir_x
            if 0 <= edge_x < width:
                return (int(edge_x), height - 1)
        
        # Top edge (y = 0)
        if dir_y < 0:
            t_top = -center_y / dir_y
            edge_x = center_x + t_top * dir_x
            if 0 <= edge_x < width:
                return (int(edge_x), 0)
        
        # Fallback: if no clean intersection found, clamp to nearest corner
        edge_x = max(0, min(width - 1, center_x + dir_x * min(width, height) / 2))
        edge_y = max(0, min(height - 1, center_y + dir_y * min(width, height) / 2))
        
        return (int(edge_x), int(edge_y))
        
    except Exception as e:
        print(f"Failed to generate edge waypoint: {e}")
        return None

## Function to visualize trajectory at a waypoint

In [12]:
def visualize_trajectory(sim, shortest_path, current_waypoint_index):
    """
    Visualize future trajectory along waypoints
    
    Parameters:
        - sim: habitat sim instance
        - shortest_path: habitat_sim.nav.ShortestPath
        - current_waypoint_index: int
    
    Returns:
        - Image at current agent position with points visualized
    """
    
    if current_waypoint_index >= len(shortest_path.points):
        return None
    
    # Get visible waypoints with their indices
    visible_waypoints_info, rgb_img = get_visible_trajectory(sim, shortest_path, current_waypoint_index)
    
    if rgb_img is None:
        return None
        
    rgb_with_points = rgb_img.copy()
    
    # Draw visible trajectory points
    for waypoint_idx, point_2d in visible_waypoints_info:
        # Color coding: green for next waypoint, yellow for future ones, blue for past ones
        if waypoint_idx == current_waypoint_index + 1:
            color = (0, 255, 0)  # Green for next waypoint
            radius = 8
        elif waypoint_idx > current_waypoint_index:
            color = (0, 255, 0)  # Yellow for future waypoints
            radius = 8
        else:
            color = (0, 255, 0)  # Blue for past waypoints
            radius = 8
        
        # Draw circle
        cv2.circle(rgb_with_points, point_2d, radius, color, -1)
        cv2.circle(rgb_with_points, point_2d, radius + 2, (0, 255, 0), 1)  # White outline
        
        # Add waypoint number
        cv2.putText(rgb_with_points, str(waypoint_idx), 
                   (point_2d[0] - 10, point_2d[1] - 15),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    # Draw trajectory lines connecting visible points
    if len(visible_waypoints_info) > 1:
        # Sort by waypoint index for proper line drawing
        sorted_waypoints = sorted(visible_waypoints_info, key=lambda x: x[0])
        
        for j in range(len(sorted_waypoints) - 1):
            waypoint_idx1, pt1 = sorted_waypoints[j]
            waypoint_idx2, pt2 = sorted_waypoints[j + 1]
            
            # Only connect consecutive or nearby waypoints
            if abs(waypoint_idx2 - waypoint_idx1) <= 3:
                cv2.line(rgb_with_points, pt1, pt2, (255, 255, 0), 2)
    
    return rgb_with_points

## Function to calculate relative rotation between two positions
So we can record where the agent faces at the end of a trajectory sequence.

This allows us to teach the VLM to generate sequences of trajectories, followed by a rotation in degrees relative to its current rotation in order to continue the path.

In [13]:
import quaternion

def calculate_relative_rotation(from_pos, to_pos, current_rotation):
    
    direction = to_pos - from_pos
    
    direction_2d = np.array([direction[0], direction[2]])
    direction_2d = direction_2d / np.linalg.norm(direction_2d)
    angle = np.arctan2(-direction_2d[0], -direction_2d[1])
    
    current_yaw = quaternion.as_euler_angles(current_rotation)[1]
    
    delta_deg = np.degrees(angle - current_yaw)
    
    return (delta_deg + 180) % 360 - 180

## PointNav Function
This function:
- Calculates the shortest path to a target position
- Navigates along the shortest path
    - From the start position, saves a visualization, the raw RGB view, and the camera coordinates of all visible waypoints
    - Jumps to the last visible point along the trajectory and repeats

In [14]:
import os
import json

def jump_to_visible_waypoints_navigation(sim, target_pos, output_dir="trajectory-vis-07282025"):
    """
    Navigate by jumping to the last visible waypoint along the trajectory.
    Saves RGB + trajectory visualizations at each jump point.
    """
    import os
    from datetime import datetime
    
    if not sim.pathfinder.is_loaded:
        print("Navigation mesh not loaded!")
        return

    agent = sim.get_agent(0)
    agent_state = agent.get_state()
    start_pos = agent_state.position
    
    shortest_path = habitat_sim.nav.ShortestPath()
    shortest_path.requested_start = start_pos
    shortest_path.requested_end = target_pos
    
    os.makedirs(output_dir, exist_ok=True)
    
    # Create a subdirectory for this run
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = os.path.join(output_dir, f"run_{timestamp}")
    os.makedirs(run_dir, exist_ok=True)
    
    if not sim.pathfinder.find_path(shortest_path):
        print("No path found!")
        return run_dir, None, None, False

    shortest_path.points = upsample_trajectory_waypoints(shortest_path)
    
    current_waypoint_idx = 0
    jump_count = 0
    jump_data = []
    
    rgb_traj_data = []
    
    success = False
    
    # Continue until we reach the final position
    while current_waypoint_idx < len(shortest_path.points) - 1:
        # Get visible waypoints with their indices
        visible_waypoints_info, rgb_img, depth_img = get_visible_trajectory(sim, shortest_path, current_waypoint_idx, return_depth_map=True)
        
        # Extract just the 2D points for visualization
        visible_points_2d = [(x, y) for _, (x, y) in visible_waypoints_info]
        
        # Create visualization
        rgb_with_trajectory = visualize_trajectory(sim, shortest_path, current_waypoint_idx)
        
        future_visible_waypoints = [idx for idx, _ in visible_waypoints_info if idx > current_waypoint_idx]
        
        final_rotation = None
        
        if future_visible_waypoints:
            last_visible_idx = max(future_visible_waypoints)

            agent_state = agent.get_state()
            current_rotation = agent_state.rotation


            if last_visible_idx + 1 < len(shortest_path.points):
                current_pos = shortest_path.points[current_waypoint_idx]
                next_waypoint_pos = shortest_path.points[last_visible_idx + 1]
                final_rotation = calculate_relative_rotation(current_pos, next_waypoint_pos, current_rotation)
        
        rgb_traj_data.append(rgb_with_trajectory.copy())
        
#         if rgb_with_trajectory is not None:
#             # Save the visualization
#             filename = f"vis_jump_{jump_count:03d}_waypoint_{current_waypoint_idx:03d}.png"
#             filepath = os.path.join(run_dir, filename)
#             cv2.imwrite(filepath, cv2.cvtColor(rgb_with_trajectory, cv2.COLOR_RGB2BGR))
            
        
        if depth_img.ndim == 3 and depth_img.shape[2] == 1:
            depth_img = depth_img[:, :, 0]

        depth_norm = cv2.normalize(depth_img, None, alpha=0, beta=255, 
                                   norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        dfilename = f"jump_{jump_count:03d}_waypoint_{current_waypoint_idx:03d}_depth.png"
        dfilepath = os.path.join(run_dir, dfilename)
        
        cv2.imwrite(dfilepath, depth_norm)
        
        rfilename = f"jump_{jump_count:03d}_waypoint_{current_waypoint_idx:03d}.png"
        rfilepath = os.path.join(run_dir, rfilename)
        cv2.imwrite(rfilepath, cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR))

        # Store jump data with waypoint indices
        visible_waypoints_data = []
        for waypoint_idx, (x, y) in visible_waypoints_info:
            waypoint_3d = shortest_path.points[waypoint_idx]
            visible_waypoints_data.append({
                "waypoint_index": int(waypoint_idx),
                "world_position": [float(waypoint_3d[0]), float(waypoint_3d[1]), float(waypoint_3d[2])],
                "image_coordinates": [int(x), int(y)]
            })

        jump_info = {
            "jump_number": jump_count,
            "rgb_filename": rfilename,
            "depth_filename": dfilename,
            "from_waypoint": current_waypoint_idx,
            "observer_position": [float(shortest_path.points[current_waypoint_idx][0]), 
                                float(shortest_path.points[current_waypoint_idx][1]), 
                                float(shortest_path.points[current_waypoint_idx][2])],
            "visible_waypoints": visible_waypoints_data,
            "total_visible": len(visible_waypoints_info),
        }

        if final_rotation is not None:
            jump_info["final_rotation"] = final_rotation

        jump_data.append(jump_info)
        
        
        if not future_visible_waypoints:
            print(f"No visible future waypoints from position {current_waypoint_idx}. Stopping.")
            break
        
        # Jump to the last visible point
        current_waypoint_idx = last_visible_idx
        jump_count += 1
        
        # Check if we've reached the end
        if current_waypoint_idx >= len(shortest_path.points) - 1:
            success = True
            break
    
    
    agent = sim.get_agent(0)
    agent_state = agent.get_state()

    
    #because we're upsampling, this puts us at most 1m away
    agent_state.position = shortest_path.points[-2]
    direction_vector = target_pos - agent_state.position
    toRotate = calculate_rotation_to_face_direction(direction_vector)
    
    agent_state.rotation = toRotate
    
    agent.set_state(agent_state)
    
    observations = sim.get_sensor_observations()
    final_img = observations["final_color"].copy()

    rgb_traj_data.append(final_img)
    
    
    metadata = {
        "timestamp": timestamp,
        "start_position": [float(start_pos[0]), float(start_pos[1]), float(start_pos[2])],
        "target_position": [float(target_pos[0]), float(target_pos[1]), float(target_pos[2])],
        "total_waypoints": len(shortest_path.points),
        "total_jumps": jump_count,
        "path_distance": float(shortest_path.geodesic_distance),
        "jumps": jump_data
    }
    
#     import json
#     metadata_file = os.path.join(run_dir, "metadata.json")
#     with open(metadata_file, 'w') as f:
#         json.dump(metadata, f, indent=2)
    
#     print(f"\nNavigation complete!")
#     print(f"Total jumps: {jump_count}")
#     print(f"Visualizations saved in: {run_dir}")
    
    return run_dir, rgb_traj_data, metadata, success


def test_jump_navigation(sim):
    # Get a random navigable target point
    target_point = sim.pathfinder.get_random_navigable_point()
    output_dir = jump_to_visible_waypoints_navigation(sim, target_point)


#test_jump_navigation(sim)

## Function to save a grid of images showing the navigation sequence
For later prompt generation.

In [15]:
import math
import matplotlib.pyplot as plt

def save_image_grid(traj_seq,
                    filepath,
                    cols=4,
                    pixel_width=2000,
                    pixel_height=2000,
                    dpi=200):
    """
    Save a list of images in a grid to disk at exactly pixel_width×pixel_height.
    """
    n = len(traj_seq)
    rows = math.ceil(n / cols)
    
    fig_w  = pixel_width  / dpi
    fig_h  = pixel_height / dpi
    
    fig, axes = plt.subplots(
        rows, cols,
        figsize=(fig_w, fig_h),
        squeeze=False
    )
    axes = axes.flatten()
    
    for idx, img in enumerate(traj_seq):
        ax = axes[idx]
        ax.imshow(img)
        ax.set_title(str(idx))
        ax.axis("off")
    
    for ax in axes[n:]:
        ax.axis("off")
    
    plt.tight_layout(pad=0)
    fig.savefig(filepath, dpi=dpi, bbox_inches="tight", pad_inches=0)
    plt.close(fig)


## ObjectNav Function
This function uses a list of object categories present in the simulator's semantic scene to get navigable points near each object, then treats the problem as a PointNav task

### Object Category List

In [16]:
object_cats = ['appliance',
 'bag',
 'barbecue',
 'basket',
 'basket of something',
 'bed',
 'bed light',
 'bed table',
 'bedside lamp',
 'book',
 'book rack',
 'cabinet',
 'candle',
 'cardboard box',
 'case',
 'chair',
 'chest',
 'clothes',
 'clothes rack',
 'computer',
 'decoration',
 'desk',
 'desk chair',
 'dining chair',
 'dining table',
 'dishwasher',
 'door',
 'door frame',
 'dresser',
 'flower vase',
 'flowerpot',
 'grill',
 'kettle',
 'kitchen cabinet',
 'kitchen counter',
 'kitchen lower cabinet',
 'lamp',
 'laptop',
 'laundry machine',
 'microwave',
 'monitor',
 'oven',
 'painting',
 'picture',
 'pillar',
 'pillow',
 'plant',
 'plunger',
 'plush toy',
 'pot',
 'printer',
 'rack',
 'refrigerator',
 'scale',
 'screen',
 'shoe',
 'sink',
 'sink cabinet',
 'sofa chair',
 'sofa seat',
 'speaker',
 'stool',
 'stovetop',
 'table',
 'tissue box',
 'tv',
 'vase',
 'wardrobe']

In [17]:
import shutil
import json


def objectNav(sim, object_category_list):
    for cat in object_category_list:
        objects = [i for i in sim.semantic_scene.objects if i.category.name() == cat]
        for obj in objects:
            
            start_point = sim.pathfinder.get_random_navigable_point()
            agent_state = habitat_sim.AgentState()
            agent_state.position = start_point
            agent_state.rotation = np.quaternion(1, 0, 0, 0)  # Identity quaternion
            sim.get_agent(0).set_state(agent_state)
            
            run_dir, traj_seq, metadata, success = jump_to_visible_waypoints_navigation(sim, obj.obb.center)#obj.aabb.center()
            
            #print(habitat_sim.AgentState().position - obj.aabb.center())
            
            if success == False:
                shutil.rmtree(run_dir)
                continue
                
            metadata['object_category'] = cat
            
            seq_filepath = os.path.join(run_dir, 'traj_seq.jpg')
            
            save_image_grid(traj_seq, seq_filepath)
            
            metadata['trajectory_sequence'] = seq_filepath
            
            metadata_file = os.path.join(run_dir, "metadata.json")
            with open(metadata_file, 'w') as f:
                json.dump(metadata, f, indent=2)
            break #Starting with just doing one object of each category to make sure we get more scene variation per 100 episodes

## Run it on a bunch of scenes

In [19]:
def build_sim(path_to_scene, path_to_config = "data/ovon/versioned_data/hm3d-0.2/hm3d/hm3d_annotated_basis.scene_dataset_config.json"):
    try:
        scene_name = path_to_scene.split('-')[-1]
        print(scene_name)
        scene_basis = f"{scene_name}.basis.glb"
        backend_cfg = habitat_sim.SimulatorConfiguration()
        backend_cfg.scene_id = os.path.join(path_to_scene, scene_basis)
        backend_cfg.scene_dataset_config_file = path_to_config
        #backend_cfg.load_semantic_mesh = True
        sensor_height = 0.5

        rgb_cfg = habitat_sim.CameraSensorSpec()
        rgb_cfg.uuid = "color"
        rgb_cfg.sensor_type = habitat_sim.SensorType.COLOR
        #rgb_cfg.hfov = mn.Deg(120)
        rgb_cfg.resolution = [1024, 1024]
        rgb_cfg.position = [0.0, sensor_height, 0.0]  # [x, y, z] relative to agent

        final_cfg = habitat_sim.CameraSensorSpec()
        final_cfg.uuid = "final_color"
        final_cfg.sensor_type = habitat_sim.SensorType.COLOR
        final_cfg.hfov = mn.Deg(120)
        final_cfg.resolution = [1024, 1024]
        final_cfg.position = [0.0, sensor_height, 0.0]  # [x, y, z] relative to agent

        # Semantic sensor
        sem_cfg = habitat_sim.CameraSensorSpec()
        sem_cfg.uuid = "semantic"
        sem_cfg.sensor_type = habitat_sim.SensorType.SEMANTIC
        #sem_cfg.hfov = mn.Deg(120)
        sem_cfg.resolution = [1024, 1024]
        sem_cfg.position = [0.0, sensor_height, 0.0]
        #

        # Depth sensor
        depth_cfg = habitat_sim.CameraSensorSpec()
        depth_cfg.uuid = "depth"
        depth_cfg.sensor_type = habitat_sim.SensorType.DEPTH
        #depth_cfg.hfov = mn.Deg(120)
        depth_cfg.resolution = [1024, 1024]
        depth_cfg.position = [0.0, sensor_height, 0.0]
        #

        agent_cfg = habitat_sim.agent.AgentConfiguration()
        agent_cfg.sensor_specifications = [rgb_cfg, sem_cfg, depth_cfg, final_cfg]

        sim_cfg = habitat_sim.Configuration(backend_cfg, [agent_cfg])
        print("Built cfg")

        sim = habitat_sim.Simulator(sim_cfg)

        navmesh_file = os.path.join(path_to_scene, f"{scene_name}.basis.navmesh")

        sim.pathfinder.load_nav_mesh(navmesh_file)
        
        return sim
    except Exception as e:
        print(f"Failed on {path_to_scene}: {e}")
        return None

In [23]:
import json

configFile = "data/ovon/versioned_data/hm3d-0.2/hm3d/hm3d_annotated_basis.scene_dataset_config.json"

with open(configFile, 'r') as f:
    configData = json.load(f)

In [24]:
scenesToUse = [i for i in configData['stages']['paths']['.glb'] if i[:5] == "train"]

In [25]:
len(scenesToUse)

145

In [25]:
scenesToUse[2:20]

['train/00016-qk9eeNeR4vw/*.basis.glb',
 'train/00017-oEPjPNSPmzL/*.basis.glb',
 'train/00020-XYyR54sxe6b/*.basis.glb',
 'train/00022-gmuS7Wgsbrx/*.basis.glb',
 'train/00023-zepmXAdrpjR/*.basis.glb',
 'train/00025-ixTj1aTMup2/*.basis.glb',
 'train/00031-Wo6kuutE9i7/*.basis.glb',
 'train/00033-oPj9qMxrDEa/*.basis.glb',
 'train/00034-6imZUJGRUq4/*.basis.glb',
 'train/00035-3XYAD64HpDr/*.basis.glb',
 'train/00043-Jfyvj3xn2aJ/*.basis.glb',
 'train/00055-HxmXPBbFCkH/*.basis.glb',
 'train/00057-1UnKg1rAb8A/*.basis.glb',
 'train/00059-kJxT5qssH4H/*.basis.glb',
 'train/00062-ACZZiU6BXLz/*.basis.glb',
 'train/00064-gQgtJ9Stk5s/*.basis.glb',
 'train/00081-5biL7VEkByM/*.basis.glb',
 'train/00087-YY8rqV6L6rf/*.basis.glb']

In [26]:
scenesToUse[16]

'train/00062-ACZZiU6BXLz/*.basis.glb'

In [27]:
hm3d_root = "data/ovon/versioned_data/hm3d-0.2/hm3d/"
scene_paths = [os.path.join(hm3d_root, i[:-12]) for i in scenesToUse[16:]]

In [28]:
scene_paths[:5]

['data/ovon/versioned_data/hm3d-0.2/hm3d/train/00062-ACZZiU6BXLz',
 'data/ovon/versioned_data/hm3d-0.2/hm3d/train/00064-gQgtJ9Stk5s',
 'data/ovon/versioned_data/hm3d-0.2/hm3d/train/00081-5biL7VEkByM',
 'data/ovon/versioned_data/hm3d-0.2/hm3d/train/00087-YY8rqV6L6rf',
 'data/ovon/versioned_data/hm3d-0.2/hm3d/train/00096-6HRFAUDqpTb']

In [ ]:
import gc

for scenepath in scene_paths[8:]:
    print(scenepath)
    sim = build_sim(scenepath)
    objectNav(sim, object_cats)
    sim.close()
    del sim
#     for i in range(3):
#         gc.collect()

data/ovon/versioned_data/hm3d-0.2/hm3d/train/00109-GTV2Y73Sn5t
GTV2Y73Sn5t
Built cfg
Renderer: NVIDIA GeForce RTX 3080 Ti/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 575.64.03
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_texture_view
    GL_ARB_framebuffer_no_attachments
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
    GL_KHR_parallel_shader_compile
    GL_NV_depth_buffer_float
Using driver workarounds:
    no-forward-compatible-core-context
    nv-egl-incorrect-gl11-function-pointers
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
 